## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Databricks Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. This data will be imported in Databricks to create the following tables under the __`country_club`__ database.

<br>
1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into the Databricks platform to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the databricks platform, but for the final deliverable,
please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

### Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [0]:
import pyspark.sql.functions as F


In [0]:
%sql 
drop database if exists country_club cascade;
create database country_club;
show databases;

databaseName
country_club
default


### Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you. 
To get started, first upload the three CSV files to the DBFS as depicted in the following figure

![](https://i.imgur.com/QcCruBr.png)


Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [0]:
# File location and type
file_location_bookings = '/FileStore/tables/Bookings.csv'
file_location_facilities = '/FileStore/tables/Facilities.csv'
file_location_members = '/FileStore/tables/Members.csv'

file_type = 'csv'

# CSV import options
infer_schema = True; first_row_is_header = True; delimiter = ','

# The applied options are for CSV files. For other file types, these will be ignored.

bookings_sdf = ( spark.read.format( file_type ) 
                                      .options( inferSchema= True, header= True, sep= delimiter ) 
                                      .load( file_location_bookings )
              )

facilities_sdf = ( spark.read.format( file_type ) 
                                  .options( inferSchema= True, header= True, sep= delimiter ) 
                                  .load( file_location_facilities )
                 )

members_sdf = ( spark.read.format(file_type) 
                                .options( inferSchema= True, header= True, sep= delimiter )
                                .load( file_location_members )
              )

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [0]:
bookings_sdf.printSchema()

root
-- bookid: integer (nullable = true)
-- facid: integer (nullable = true)
-- memid: integer (nullable = true)
-- starttime: string (nullable = true)
-- slots: integer (nullable = true)

In [0]:
print( 'Bookings Schema -->'); print( bookings_sdf.printSchema() )
print( 'Facilities Schema -->'); print( facilities_sdf.printSchema() )
print( 'Members Schema -->'); print( members_sdf.printSchema() )

Bookings Schema -->
root
-- bookid: integer (nullable = true)
-- facid: integer (nullable = true)
-- memid: integer (nullable = true)
-- starttime: string (nullable = true)
-- slots: integer (nullable = true)

None
Facilities Schema -->
root
-- facid: integer (nullable = true)
-- name: string (nullable = true)
-- membercost: double (nullable = true)
-- guestcost: double (nullable = true)
-- initialoutlay: integer (nullable = true)
-- monthlymaintenance: integer (nullable = true)

None
Members Schema -->
root
-- memid: integer (nullable = true)
-- surname: string (nullable = true)
-- firstname: string (nullable = true)
-- address: string (nullable = true)
-- zipcode: integer (nullable = true)
-- telephone: string (nullable = true)
-- recommendedby: integer (nullable = true)
-- joindate: string (nullable = true)

None

### Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [0]:
bookings_sdf.write.format( source= 'parquet' ).saveAsTable( 'country_club.Bookings' )
facilities_sdf.write.format( source= 'parquet' ).saveAsTable( 'country_club.Facilities' )
members_sdf.write.format( source= 'parquet' ).saveAsTable( 'country_club.Members' )

### Refresh tables and check them

In [0]:
%sql
use country_club;
REFRESH table bookings; REFRESH table facilities; REFRESH table members;
show tables;

database,tableName,isTemporary
country_club,bookings,false
country_club,facilities,false
country_club,members,false


### Test a sample SQL query

__Note:__ You can use __`%sql`__ at the beginning of a cell and write SQL queries directly as seen in the following cell. Neat isn't it!

In [0]:
%sql
select * from bookings limit 3

bookid,facid,memid,starttime,slots
0,3,1,2012-07-03 11:00:00,2
1,4,1,2012-07-03 08:00:00,2
2,6,0,2012-07-03 18:00:00,2


#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [0]:
%sql
select * from facilities
where membercost = 0

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
2,Badminton Court,0.0,15.5,4000,50
3,Table Tennis,0.0,5.0,320,10
7,Snooker Table,0.0,5.0,450,15
8,Pool Table,0.0,5.0,400,15


####  Q2: How many facilities do not charge a fee to members?

In [0]:
%sql
select count( name )
from facilities
  where membercost = 0

count(name)
4


In [0]:
facilities_sdf.show(3)

+-----+---------------+----------+---------+-------------+------------------+
facid| name|membercost|guestcost|initialoutlay|monthlymaintenance|
+-----+---------------+----------+---------+-------------+------------------+
 0| Tennis Court 1| 5.0| 25.0| 10000| 200|
 1| Tennis Court 2| 5.0| 25.0| 8000| 200|
 2|Badminton Court| 0.0| 15.5| 4000| 50|
+-----+---------------+----------+---------+-------------+------------------+
only showing top 3 rows

#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [0]:
facilities_sdf[ (facilities_sdf.membercost+facilities_sdf.guestcost) < 20./100*facilities_sdf.monthlymaintenance ].show()

+-----+--------------+----------+---------+-------------+------------------+
facid| name|membercost|guestcost|initialoutlay|monthlymaintenance|
+-----+--------------+----------+---------+-------------+------------------+
 0|Tennis Court 1| 5.0| 25.0| 10000| 200|
 1|Tennis Court 2| 5.0| 25.0| 8000| 200|
 4|Massage Room 1| 9.9| 80.0| 4000| 3000|
 5|Massage Room 2| 9.9| 80.0| 4000| 3000|
+-----+--------------+----------+---------+-------------+------------------+

#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [0]:
%sql
select *
  from facilities
  where facid in ( 1,5 )

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
1,Tennis Court 2,5.0,25.0,8000,200
5,Massage Room 2,9.9,80.0,4000,3000


#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [0]:
import pyspark.sql.functions as F

facilities_sdf.withColumn( 'cost_label', col= F.when( facilities_sdf.monthlymaintenance > 100, 'expensive' )\
                                               .otherwise( 'cheap' )                         
                         ).show()

+-----+---------------+----------+---------+-------------+------------------+----------+
facid| name|membercost|guestcost|initialoutlay|monthlymaintenance|cost_label|
+-----+---------------+----------+---------+-------------+------------------+----------+
 0| Tennis Court 1| 5.0| 25.0| 10000| 200| expensive|
 1| Tennis Court 2| 5.0| 25.0| 8000| 200| expensive|
 2|Badminton Court| 0.0| 15.5| 4000| 50| cheap|
 3| Table Tennis| 0.0| 5.0| 320| 10| cheap|
 4| Massage Room 1| 9.9| 80.0| 4000| 3000| expensive|
 5| Massage Room 2| 9.9| 80.0| 4000| 3000| expensive|
 6| Squash Court| 3.5| 17.5| 5000| 80| cheap|
 7| Snooker Table| 0.0| 5.0| 450| 15| cheap|
 8| Pool Table| 0.0| 5.0| 400| 15| cheap|
+-----+---------------+----------+---------+-------------+------------------+----------+

#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [0]:
import pyspark.sql.functions as F

members_sdf = members_sdf.withColumn(  'dt', F.to_timestamp( members_sdf.joindate, format= 'yyyy-MM-dd HH:mm:ss' )  )

members_sdf.filter( members_sdf.dt == members_sdf.select( F.max('dt') ).collect()[0][0] ).select( 'firstname','surname','dt' ).show()

+---------+-------+-------------------+
firstname|surname| dt|
+---------+-------+-------------------+
 Darren| Smith|2012-09-26 18:08:45|
+---------+-------+-------------------+

####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [0]:
# selecting and concanating first and last name
memberName_sdf = members_sdf.select(   members_sdf.memid, F.concat_ws( ' ', members_sdf.firstname,members_sdf.surname ).alias('mem_name')  )

facil_sdf = facilities_sdf.select(  facilities_sdf.facid, facilities_sdf.name.alias('facility_name')  )

# joining all 3 tables

comb_sdf = bookings_sdf.select( ['bookid','facid','memid'] ) \
                           .join( memberName_sdf, on= 'memid' ) \
                           .join( facil_sdf, on= 'facid' )

In [0]:
# member who have used Tennis Court
comb_sdf.filter( comb_sdf.facility_name.contains('Tennis') )\
        .select('mem_name').distinct().sort('mem_name').show()

+--------------------+
 mem_name|
+--------------------+
 Anna Mackenzie|
 Anne Baker|
 Burton Tracy|
 Charles Owen|
 Darren Smith|
 David Farrell|
 David Jones|
 David Pinker|
 Douglas Jones|
 Erica Crumpet|
 Florence Bader|
 GUEST GUEST|
 Gerald Butters|
 Henrietta Rumney|
Henry Worthington...|
 Jack Smith|
 Janice Joplette|
 Jemima Farrell|
 Joan Coplin|
 John Hunt|
+--------------------+
only showing top 20 rows

In [0]:
combConcernedDt_sdf.show(4)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1642975552791105> in <module> 
 ----> 1 combConcernedDt_sdf . show ( 4 ) 

 NameError : name 'combConcernedDt_sdf' is not defined

#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [0]:
memberName_sdf
# subset col 
facility_sdf = facilities_sdf.select( ['facid','name','membercost','guestcost']  )

bookings_sdf = bookings_sdf.withColumn(  'startTime_dt', F.to_timestamp( bookings_sdf.starttime, format= 'yyyy-MM-dd HH:mm:ss' )  )

# join 3 tables
comb_sdf = bookings_sdf.join( memberName_sdf, on= 'memid' ).join( facility_sdf, on= 'facid' ).drop('starttime')

# subset rows for concerned date
dt_sdf = comb_sdf.filter(  F.to_date(comb_sdf.startTime_dt) == F.lit('2012-09-14')  )

In [0]:
# separating guest and member cost for calcualtion
guest_cost = dt_sdf.filter(  dt_sdf.mem_name.contains('GUEST GUEST')  ) \
                   .withColumn( 'total_cost', dt_sdf.guestcost*dt_sdf.slots  )

member_cost = dt_sdf.filter(  ~dt_sdf.mem_name.contains('GUEST GUEST')  ) \
                   .withColumn( 'total_cost', dt_sdf.membercost*dt_sdf.slots  )

totalCost_sdf = member_cost.union( guest_cost )

totalCost_more30_sdf = totalCost_sdf.filter( totalCost_sdf.total_cost > 30 ).sort( 'total_cost', ascending= False )
totalCost_more30_sdf.show()

+-----+-----+------+-----+-------------------+--------------+--------------+----------+---------+----------+
facid|memid|bookid|slots| startTime_dt| mem_name| name|membercost|guestcost|total_cost|
+-----+-----+------+-----+-------------------+--------------+--------------+----------+---------+----------+
 5| 0| 2946| 4|2012-09-14 11:00:00| GUEST GUEST|Massage Room 2| 9.9| 80.0| 320.0|
 4| 0| 2940| 2|2012-09-14 13:00:00| GUEST GUEST|Massage Room 1| 9.9| 80.0| 160.0|
 4| 0| 2937| 2|2012-09-14 09:00:00| GUEST GUEST|Massage Room 1| 9.9| 80.0| 160.0|
 4| 0| 2942| 2|2012-09-14 16:00:00| GUEST GUEST|Massage Room 1| 9.9| 80.0| 160.0|
 1| 0| 2926| 6|2012-09-14 17:00:00| GUEST GUEST|Tennis Court 2| 5.0| 25.0| 150.0|
 0| 0| 2922| 3|2012-09-14 19:00:00| GUEST GUEST|Tennis Court 1| 5.0| 25.0| 75.0|
 1| 0| 2925| 3|2012-09-14 14:00:00| GUEST GUEST|Tennis Court 2| 5.0| 25.0| 75.0|
 0| 0| 2920| 3|2012-09-14 16:00:00| GUEST GUEST|Tennis Court 1| 5.0| 25.0| 75.0|
 6| 0| 2948| 4|2012-09-14 09:30:00| GUEST GUEST| Squash Court| 3.5| 17.5| 70.0|
 4| 13| 2941| 4|2012-09-14 14:00:00|Jemima Farrell|Massage Room 1| 9.9| 80.0| 39.6|
 6| 0| 2949| 2|2012-09-14 12:30:00| GUEST GUEST| Squash Court| 3.5| 17.5| 35.0|
 6| 0| 2951| 2|2012-09-14 15:00:00| GUEST GUEST| Squash Court| 3.5| 17.5| 35.0|
+-----+-----+------+-----+-------------------+--------------+--------------+----------+---------+----------+

#### Q9: This time, produce the same result as in Q8, but using a subquery.

#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [0]:
totalCost_df.show(3)

+-----+-----+------+--------------+------------+----------+---------+----------+
facid|memid|bookid| name| mem_name|membercost|guestcost|total_cost|
+-----+-----+------+--------------+------------+----------+---------+----------+
 3| 1| 0| Table Tennis|Darren Smith| 0.0| 5.0| 0.0|
 4| 1| 1|Massage Room 1|Darren Smith| 9.9| 80.0| 19.8|
 6| 0| 2| Squash Court| GUEST GUEST| 3.5| 17.5| 35.0|
+-----+-----+------+--------------+------------+----------+---------+----------+
only showing top 3 rows

In [0]:
comb_sdf = comb_sdf.withColumn( 'total_cost',  F.when( comb_sdf.memid == 0, comb_sdf.slots*comb_sdf.guestcost ) \
                                                .otherwise( comb_sdf.slots*comb_sdf.membercost )
                              )

totalCost_df = comb_sdf.select( ['facid','memid','bookid','name','mem_name','membercost','guestcost','total_cost'] )
totCost_grpBy = totalCost_df.groupBy( ['name'] ).agg( {'total_cost':'sum'} ).sort( 'sum(total_cost)' )

totCost_grpBy.filter( totCost_grpBy['sum(total_cost)']<1000 ).show()

+-------------+---------------+
 name|sum(total_cost)|
+-------------+---------------+
 Table Tennis| 180.0|
Snooker Table| 240.0|
 Pool Table| 270.0|
+-------------+---------------+